In [1]:
import pandas as pd
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# from spotipy.oauth2 import SpotifyOAuth
import json
import time
import sys
import base64
import numpy as np
import requests
import datetime
import os
from urllib.parse import urlencode
os.chdir("C:\\Users\\ip3064\\Desktop\\GitHub\\Spotify-API")

In [ ]:
#Spotify Devl Credentials
client_id = ''
client_secret = ''

base_url = 'https://api.spotify.com/v1/'

In [ ]:
#Class containing methods to extract data from Spotify End points and perform some data cleansing

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    albs_dict = {}
    

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64


    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "authorization": f"Basic {client_creds_b64.decode()}" 
    }

    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
    }


    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(token_url, data=token_data, headers=token_header)
        if r.status_code not in range(200,299):
            # return False
            raise Exception("Could not Peform Authentication")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data["access_token"]
        self.access_token = access_token
        expires_in = data["expires_in"] #Seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers

    
    def get_resource(self, lookup_id, resource_type='albums',version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    
    def get_artist(self, _id):
        return self.get_resource(lookup_id=_id, resource_type='artists')

    
    def get_album(self, _id):
        return self.get_resource(lookup_id=_id, resource_type='albums')


    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if(query==None):
            raise Exception("Please enter a query to search")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k, v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "and":
                operator = operator.upper()
                if isinstance(operator, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        return self.base_search(query_params)

    def track_artist_id_search(self, query):
        try:
            t = spotify.search(query=query,search_type="track")
            h = t['tracks']['items']
            art_name = h[0]['album']['artists'][0]['name']
            art_id = h[0]['album']['artists'][0]['id']
            print(f"Artist name: {art_name}, Artist ID: {art_id}")
            artist = {art_name:art_id}
            return artist
        except:
            print(f"Bad Serach Query, Error:{sys.exc_info()[0]} ... Try Again")

    def artist_artist_id_search(self, query):
        try:
            a = spotify.search(query=query,search_type="artist")
            artist_id = a['artists']['items'][0]['id']
            artist={query:artist_id}
            print(f"Artist name: {query}, Artist ID: {artist_id}")
            return artist
        except:
            print(f"Bad Serach Query, Error:{sys.exc_info()[0]} ... Try Again")

    
    def artist_id_search(self, query, search_type="artist"):
        if search_type.lower() == 'artist':
            artist = self.artist_artist_id_search(query)
        elif search_type.lower() == 'track':
            artist = self.track_artist_id_search(query)
        else:
            raise Exception("Please enter a valid artist name or track to search")
            return {}
        return artist

    
    def fetch_albums_df(self, albs, clean='N'):
        albums = albs['items']
        album_names=[]
        d = []
        for i in range(len(albums)):
            album_name = albums[i]['name']
            trim_name = album_name.split('(')[0].strip()
            if trim_name.upper() in album_names:
                continue
            else:
                album_names.append(trim_name)

                alb_dict = {
                    'album_name': albums[i]['name'],
                    'album_short_name': trim_name,
                    'artist': albums[i]['artists'][0]['name'],
                    'release_date': albums[i]['release_date'],
                    'total_tracks': albums[i]['total_tracks'],
                    'album_id': albums[i]['id'],
                    'album_uri': albums[i]['uri'],
                    'next_link': albs['next']
            }

            d.append(alb_dict)

        df = pd.DataFrame(d)
        return df

    
    def fetch_albums(self, artist_id, offset=0,limit=50,grp='album'):
        headers = self.get_resource_header()
        endpoint = base_url + 'artists/' + artist_id + '/albums'

        r = requests.get(endpoint,
                        headers = headers,
                        params = {'include_groups':grp,'limit':limit,'offset':offset})

        albs = r.json()  
        
        return albs


    def get_artist_albums(self, query,limit=50,search_type='artist'):
        """
        Confirm artist and artist id and invoke album fetch
        """
        artist = self.artist_id_search(query=query,search_type=search_type)
        for key, value in artist.items():
            print(f"Artist: {key}, Artist ID: {value}")
            artist_id = value

        artist_id_check = input(f'Artist Name: {key}, Confirm - Y/N?')
        if artist_id_check == 'Y':
            print(f'Fetching {key}\'s albums')
            albs_dict = self.fetch_albums(artist_id) #limit extension?
            self.albs_dict = albs_dict
            albs_df = self.fetch_albums_df(albs_dict)
        elif artist_id_check == 'N':
            print('Retry search with artist name or any one of their popular tracks')
        else:
            print('Invalid Entry, try again')

        return albs_df
        
          

In [ ]:
#Creating an instance of the class

spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
#Method to get Top 50 albums of an artist

albs_df = spotify.get_artist_albums('A R Rahman')